# DRMM (Deep Relevance Matching Model)

_Ismaël Bonneau_

Mots-clés: _Relevance Matching_, _Semantic Matching_, _Neural Models_,
_Ad-hoc Retrieval_, _Ranking Models_


#### But de ce notebook: Comprendre et construire une architecture **DRMM** fonctionnelle avec **pytorch**, et l'expliquer de façon concise.

Un gros bisou à Daniel Godoy pour son tutoriel <a href="https://towardsdatascience.com/understanding-pytorch-with-an-example-a-step-by-step-tutorial-81fc5f8c4e8e">comprendre pytorch par l'exemple pas à pas</a> (en Anglais).

Pour cela, 2 étapes:

- construire la chaîne de pré traitements:
    - générer des paires document-requête non pertinentes et pertinentes pour l'apprentissage
    - générer des histogrammes d'interaction locales au niveau document-requête
- construire l'architecture DRMM

Les interractions sont pour le moment des interactions locales sur des word embeddings et sont mesurées comme une similarité cosinus entre les vecteurs des mots de la requête et ceux du document.

## En quoi consiste le modèle DRMM?

lien vers <a href="https://arxiv.org/pdf/1711.08611.pdf">l'article</a> (en Anglais) par _Jiafeng Guo_, _Yixing Fan_, _Qingyao Ai_, _W. Bruce Croft_ (2017) [1]

DRMM (Deep Relevance Matching Model) est un modèle de réseau de neurones profond pour la RI (recherche d'information).

Un des objectifs principaux de la RI est de déterminer la **pertinence** d'un document (cela peut être un document court sous forme d'un paragraphe, ou long, voire très long) par rapport à une requête donnée. Un moteur de RI traditionnel retournera alors une liste ordonnée des documents par pertinence par rapport à une requête posée par l'utilisateur, avec en tête de liste les documents les plus pertinents.

Un problème qui se pose avec de nombreux modèles de RI est . Certains termes de la requête ne se trouvent pas toujours dans des documents qui sont pourtant pertinents pour cette requête. Pensons à une requête sur la pigeons dans la ville de Paris: Un document qui aurait pour sujet le "problème envahissant des oiseaux dans la capitale Française", sans contenir une seule fois les mots Pigeons et Paris, aurait peu de chance d'être considéré comme pertinent.

Plusieurs solutions sont donc proposées pour résoudre ce problème.

DRMM est un modèle **orienté interactions**, qui applique une fonction apprise (un réseau de neurones n'est rien d'autres qu'une fonction très complexe) à des interactions entre un document et une requête, qui ne font donc pas partie du réseau et ne sont pas apprises. Cette fonction a pour but de calculer un **score pour la paire document-requête**, score d'autant **plus élevé que le document est pertinent pour la requête**.

Il se compose de deux parties: 

- Une partie **"feed forward matching network"** qui est un simple perceptron multicouche. Il s'agit dans l'implémentation de [1] d'un perceptron à 3 couches, de taille 30 neurones, 5 neurones, et 1 neurone. Cette partie vient calculer un score pour l'interaction de chaque terme de la requête avec l'ensemble des termes du document. Pour une requête de 5 termes, le partie feed forward matching produira donc un vecteur de dimension 5, pour 1 score par terme de la requête. On a donc un bloc qui prend en entrée une matrice d'interactions des termes de la requête, et qui rend en sortie un vecteur de score pour chaque terme de la requête.

- Une partie **"term gating network"** qui est un perceptron à une couche. Il s'agit uniquement d'apprendre un vecteur qui vient mutiplier chaque "vecteur de terme" de la requête et ensuite pondérer les scores calculés par la partie feed forward. 

<img src="images/DRMMschema.png" width="700"> 

## Dataset

On utilise pour cet exemple le dataset <a href="https://trec.nist.gov/data/robust/04.guidelines.html">robust 2004</a>. Il s'agit d'une collection de documents datés de 2004 provenant de journaux américains, le LA times, le Financial times, le Federal Register 94, et le Foreign Broadcast Information Service.


| nombre de requêtes      |  jugements de pertinence    |
| -------------:|  ------------- |
|     **250**       |   **311,410**

| collection      |     nombre de documents    |     taille en Mo    |
| ------------- |: -------------: |: -------------: |
| Financial times      |        210 158        |        564        |
| La times       |        131,896        |        475        |
| FBIS      |        130,471        |        470        |
| FR94      |        55,630        |        395        |
| **TOTAL**      |        **528,155**        |        **1904**        |

In [15]:
import numpy as np
import matplotlib.pyplot as plt
from os import sep
import os
import random
import pickle

import torch

%matplotlib inline
%load_ext autoreload
%autoreload 2

embeddings_path = "embeddings"
dataset_path = "data"

## Pré traitements: 

### Obtenir des word embeddings 

Ce word embedding a les caractéristiques suivantes:

- Gensim word2vec Continuous Skipgram
- taille de vecteur ${300}$
- window ${10}$
- entrainé sur la collection robust 2004
- lemmatisation avec Krovetz, mots en minuscule
- ${116832}$ mots

voir script <a href="https://github.com/ismaelbonneau/semantic_deep_neuralIR/blob/master/scripts/generate_embeddings.py">generate_embeddings.py</a>

In [1]:
import gensim
word_vectors = gensim.models.Word2Vec.load('embeddings/model_1')
word_vectors.init_sims(True)

In [13]:
word_vectors.wv.most_similar("toulouse")[:5] # c'est nooooooormal la mif

[('albi', 0.6107227802276611),
 ('montpellier', 0.5868463516235352),
 ('grenoble', 0.537689745426178),
 ('marseille', 0.5351033210754395),
 ('tarbe', 0.5294458866119385)]

### On récupère les requêtes:

Elles se trouvent sous forme de tuple ([mots clés], [texte de la requête]). On ne garde que les mots clés.

In [82]:
import ast
from gensim.parsing.preprocessing import preprocess_string, strip_punctuation
from krovetzstemmer import Stemmer #stemmer pas mal pour la PR
ks = Stemmer()

def clean(txt):
    return " ".join([ks.stem(t) for t in txt.replace(",", "").replace(".", "").split(" ")])   
with open(dataset_path + sep + "robust2004.txt", "r") as f:
    queries = ast.literal_eval(f.read())
queries = {d:clean(queries[d][0]) for d in queries}

print(queries["307"])
print(queries["404"])

new hydroelectric project
ireland peace talks


### Le DRMM a deux entrées: une entrée interactions et une entrée termes.

L'entrée termes prend un vecteur d'idf des termes de la requête. Cette information est sensée aider à pondérer les termes de la requête en fonction de leur importance. Un IDF élevé indique un mot "rare" dans le corpus, donc probablement important pour la requête. Il faut donc pouvoir récupérer efficacement des **idf**. Pour cela, on construit un dictionnaire terme -> idf qui nous servira dans l'étape d'après.

voir script <a href="https://github.com/ismaelbonneau/semantic_deep_neuralIR/blob/master/scripts/get_idf.py">get_idf.py</a>

In [84]:
#bon là on charge du coup vu que le fichier est sauvegardé sur le disque
idf = pickle.load(open("idf_robust2004.pkl", "rb"))
print("idf Paris: ", idf["paris"])
print("idf Toulouse: ", idf["toulouse"])

idf Paris:  4.317922943975862
idf Toulouse:  8.631967635734412


### On prépare dans des fichiers les matrices d'interactions et les vecteurs d'idf 

In [9]:
from datasets import Robust04 #permet de gérer le chargement et le traitement de robust2004

inputgenerator = Robust04(intervals=30, model_wv=word_vectors)
inputgenerator.load_idf(idf_file="idf_robust2004.pkl")
inputgenerator.load_all_query(file_query="data/robust2004.txt")
inputgenerator.load_relevance(file_rel="data/qrels.robust2004.txt")
inputgenerator.load_all_docs()

query chargé
relevance chargé
docs chargés


#### Cette méthode calcule les matrices d'interraction et les charge dans des fichiers

In [ ]:
inputgenerator.prepare_data_forNN("saved_data")

#### Cette méthode pré calcule les matrices d'interraction des résultats de BM25 et les charge dans des fichiers

In [12]:
inputgenerator.prepare_data_reranking(pickle.load(open("results_bm25_robust.pkl", "rb")))

nombre de requetes: 249.
data completed


### Regardons à quoi ressemblent les histogrammes d'interraction:

<img src="images/avgrelhist307.png">
<img src="images/avgnonrelhist307.png">
<img src="images/avgrelhist404.png">
<img src="images/avgnonrelhist404.png">

On peut voir que les documents jugés "relevant" montrent de plus grandes interractions sur l'ensemble des termes.

## Architecture du modèle

### Avec pytorch

L'implémentation du modèle est très simple: il s'agit d'un MLP à 2 couches et d'un vecteur pour le termgating (une matrice si on choisit de considérer les embeddings des termes au lieu de l'idf)

In [40]:
#hérite de la classe Pytorch Module
class DRMM(torch.nn.Module):
    def __init__(self, hist_size, query_term_maxlen, hidden_sizes=[5,1], use_cuda=True):
        super(DRMM, self).__init__()
        self.mlp = nn.Sequential(nn.Linear(hist_size, hidden_sizes[0]), nn.Tanh(), 
            nn.Linear(hidden_sizes[0], hidden_sizes[1]), nn.Tanh())
        # term gating (scalar)
        self.termgating = torch.nn.Parameter(torch.rand(1), requires_grad=True)
    
    def forward(self, interractions, termvector):
        """
        interractions: (query_term_maxlen, hist_size)
        termvector: (1, query_term_maxlen)
        """
        #partie histogramme
        interractions_output = self.mlp(interractions).squeeze()
        # partie term gating
        gating_output = torch.nn.functional.softmax((self.termgating * termvector).squeeze(), dim=1)
        #combiner les 2 avec un produit scalaire
        axis = 1
        s = torch.sum(gating_output * interractions_output, dim = axis)
        return s

### Fonction de coût à optimiser: **Margin Ranking Hinge Loss**

Objectif: Pour une requête, étant donné deux documents ${d^+}$ et ${d^-}$ tels que ${d^+}$ doit être ordonné avant ${d^-}$ dans les résultats de pertinence, cette loss cherche à attribuer un score plus élevé à ${d^+}$ qu'à ${d^-}$, de sorte qu'il soit au moins égal à une certaine marge (1, dans notre cas).

Cette fonction peut aussi être vue comme une fonction qui cherche à minimiser le nombre de paires mal ordonnées dans la liste finale!

${rankingHingeLoss(d^+, d^-) = max(0,1 - s(d^+) + s(d^-))}$ où ${s(d^+)}$ est le score de ${d^+}$.

In [ ]:
from torch.nn import MarginRankingLoss

### On va créer une classe de Dataset qui contient les paires pertinentes et les paires non pertinentes.

Plus précisément, la classe Dataset de Pytorch est un wrapper pour X (traditionnellement les vecteurs de features) et Y (traditionnellement les labels). Ici, on a pas besoin de vecteur de labels. On peut donc transformer l'utilisation de la classe pour mettre dans X une matrice de paires (interractions relevant, vecteur de termes relevant) et dans Y la même matrice, pour les paires non pertinentes. L'avantage est que lors du shuffle avec le DataLoader, on conservera l'alignement doc pertinent/non pertinent pour une requête et on ne sera pas embêté pour utiliser la ranking hinge loss.

In [20]:
from torch.utils.data import Dataset

class DrmmDataset(Dataset):
    def __init__(self, pos_tensor, neg_tensor):
        self.x = pos_tensor
        self.y = neg_tensor
    def __getitem__(self, index):
        return (self.x[index], self.y[index])
    def __len__(self):
        return len(self.x)

### On peut maintenant tester le modèle avec des métriques de RI:

Pour chacune des requêtes de test, on récupère les 2000 premiers documents retournés par *BM25*. Nous procédons en réordonnant cette liste de documents avec notre modèle *DRMM* appris, et nous calculons des performances sur les 1000 premiers documents de ce re ranking sur un ensemble de métriques de RI (**NDCG@20**, **MAP**, **P@20**), en le comparant avec le ranking original *BM25*.

Pour BM25, l'indexation a été faite comme suit:

- indexation sur le texte du document et non le titre
- texte sans stopwords, ponctuation, balises
- stemming avec Krovetz

<img src="images/learning_to_rank.png">

## Bibliographie


[1] <a href="http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.118.161&rep=rep1&type=pdf">What Works Better for Question Answering: Stemming or
Morphological Query Expansion?</a>

[2] <a href="https://dl.acm.org/citation.cfm?id=160718" >Viewing morphology as an inference process</a>

[3] <a href="https://arxiv.org/pdf/1809.01682.pdf">Deep Relevance Ranking Using Enhanced Document-Query Interactions</a>

[4] <a href="http://delivery.acm.org/10.1145/2810000/2806475/p1411-kenter.pdf?ip=132.227.125.83&id=2806475&acc=ACTIVE%20SERVICE&key=7EBF6E77E86B478F%2EA72B4D473219EA0C%2E4D4702B0C3E38B35%2E4D4702B0C3E38B35&__acm__=1562588994_ca050bde6ad37b2c0d433db2e5df63ba">Short Text Similarity with Word Embeddings</a>